In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import is_bool_dtype, is_object_dtype,is_categorical_dtype
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import random
!pip install category_encoders
import category_encoders as ce
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
from PIL import  Image
%matplotlib inline
import seaborn as sns
import itertools
from sklearn.metrics import classification_report

In [ ]:
Data = pd.read_excel('Edison Financial Data Science Project Clean.xlsx')
Data.head(5)

,JumpstartFlag,IsFirstTimeBuyer__c,LoanAmount__c,Loan Amount (50k Bins),LoanPurpose__c,SelfAssessedCreditRating__c,Source,LeadDate,LeadDate_Binary,Lead_to_Allocate,...,Lead_to_Credit,Allocate_to_Credit,Credit FirstStatusDate,Credit FirstStatusDate_Binary,Lead_to_Submit,Credit_to_Submit,SubmitToLender FirstStatusDate,SubmitToLender FirstStatusDate_Binary,FileComplete FirstStatusDate,FileComplete FirstStatusDate_Binary
0,0,False,452500.0,350000.0,Refinance,NaN,LowestRates,2020-08-09,1,1.0,...,2.0,1.0,2020-08-11,1,5.0,3.0,2020-08-14,1,2020-09-24,1
1,0,False,908000.0,900000.0,Purchase,NaN,LowestRates,2020-08-03,1,8.0,...,-1.0,-1.0,NaT,0,-1.0,-1.0,NaT,0,NaT,0
2,0,False,65000.0,50000.0,Refinance,NaN,Lower My Bills,2021-04-08,1,78.0,...,-1.0,-1.0,NaT,0,-1.0,-1.0,NaT,0,NaT,0
3,0,False,160000.0,150000.0,Purchase,Good,Canada Mortgage Leads,2022-01-17,1,-1.0,...,-1.0,-1.0,NaT,0,-1.0,-1.0,NaT,0,NaT,0
4,0,False,800000.0,800000.0,Purchase,Average,Canada Mortgage Leads,2022-01-17,1,-1.0,...,-1.0,-1.0,NaT,0,-1.0,-1.0,NaT,0,NaT,0


In [ ]:
def Info(df):
    with pd.option_context("display.max_colwidth", 20):
        info = pd.DataFrame()
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        info['No. unique'] = df.apply(lambda x: len(x.unique()))
        info['unique values'] = df.apply(lambda x: x.unique())
        return info.sort_values('percent missing')

Info(Data)

,data type,percent missing,No. unique,unique values
JumpstartFlag,int64,0.000000,2,"[0, 1]"
SubmitToLender FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
Credit FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
AllocateDate_Binary,int64,0.000000,2,"[1, 0]"
LeadDate_Binary,int64,0.000000,1,[1]
LeadDate,datetime64[ns],0.000000,700,"[2020-08-09T00:00:00.000000000, 2020-08-03T00:..."
FileComplete FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
LoanPurpose__c,object,0.000000,4,"[Refinance, Purchase, Renewal, Unknown]"
IsFirstTimeBuyer__c,bool,0.000000,2,"[False, True]"
Credit_to_Submit,float64,0.012629,184,"[3.0, -1.0, 1.0, 14.0, 21.0, 4.0, 10.0, 2.0, 1..."


In [ ]:
Data_Clean = Data.select_dtypes(include=['object','float64','bool','int64'])
Info(Data_Clean)

,data type,percent missing,No. unique,unique values
JumpstartFlag,int64,0.000000,2,"[0, 1]"
Credit FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
AllocateDate_Binary,int64,0.000000,2,"[1, 0]"
SubmitToLender FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
LeadDate_Binary,int64,0.000000,1,[1]
FileComplete FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
LoanPurpose__c,object,0.000000,4,"[Refinance, Purchase, Renewal, Unknown]"
IsFirstTimeBuyer__c,bool,0.000000,2,"[False, True]"
Credit_to_Submit,float64,0.012629,184,"[3.0, -1.0, 1.0, 14.0, 21.0, 4.0, 10.0, 2.0, 1..."
Lead_to_Submit,float64,0.014733,176,"[5.0, -1.0, 74.0, 14.0, 22.0, 7.0, 13.0, 2.0, ..."


In [ ]:
Data_Clean=Data_Clean.dropna(subset=['Credit_to_Submit'])
Data_Clean=Data_Clean.dropna(subset=['Lead_to_Submit'])
Data_Clean=Data_Clean.dropna(subset=['Allocate_to_Credit'])
Data_Clean=Data_Clean.dropna(subset=['Lead_to_Credit'])
Data_Clean=Data_Clean.dropna(subset=['Lead_to_Allocate'])
Data_Clean=Data_Clean.dropna(subset=['Loan Amount (50k Bins)'])
Data_Clean=Data_Clean.dropna(subset=['LoanAmount__c'])
Info(Data_Clean)

,data type,percent missing,No. unique,unique values
JumpstartFlag,int64,0.000000,2,"[0, 1]"
Credit_to_Submit,float64,0.000000,167,"[3.0, -1.0, 1.0, 14.0, 21.0, 4.0, 10.0, 2.0, 1..."
Lead_to_Submit,float64,0.000000,173,"[5.0, -1.0, 74.0, 14.0, 22.0, 7.0, 13.0, 2.0, ..."
Credit FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
Allocate_to_Credit,float64,0.000000,110,"[1.0, -1.0, 0.0, 2.0, 5.0, 41.0, 3.0, 23.0, 10..."
Lead_to_Credit,float64,0.000000,302,"[2.0, -1.0, 1.0, 73.0, 8.0, 3.0, 4.0, 0.0, 9.0..."
AllocateDate_Binary,int64,0.000000,2,"[1, 0]"
SubmitToLender FirstStatusDate_Binary,int64,0.000000,2,"[1, 0]"
Lead_to_Allocate,float64,0.000000,564,"[1.0, 8.0, 78.0, -1.0, 10.0, 11.0, 7.0, 3.0, 2..."
LoanPurpose__c,object,0.000000,4,"[Refinance, Purchase, Renewal, Unknown]"


In [ ]:
Data_Clean['Source']=Data_Clean['Source'].fillna('Unknown')
Data_Clean['SelfAssessedCreditRating__c']=Data_Clean['SelfAssessedCreditRating__c'].fillna('NotReported')
Info(Data_Clean)

,data type,percent missing,No. unique,unique values
JumpstartFlag,int64,0.0,2,"[0, 1]"
Credit_to_Submit,float64,0.0,167,"[3.0, -1.0, 1.0, 14.0, 21.0, 4.0, 10.0, 2.0, 1..."
Lead_to_Submit,float64,0.0,173,"[5.0, -1.0, 74.0, 14.0, 22.0, 7.0, 13.0, 2.0, ..."
Credit FirstStatusDate_Binary,int64,0.0,2,"[1, 0]"
Allocate_to_Credit,float64,0.0,110,"[1.0, -1.0, 0.0, 2.0, 5.0, 41.0, 3.0, 23.0, 10..."
Lead_to_Credit,float64,0.0,302,"[2.0, -1.0, 1.0, 73.0, 8.0, 3.0, 4.0, 0.0, 9.0..."
AllocateDate_Binary,int64,0.0,2,"[1, 0]"
SubmitToLender FirstStatusDate_Binary,int64,0.0,2,"[1, 0]"
Lead_to_Allocate,float64,0.0,564,"[1.0, 8.0, 78.0, -1.0, 10.0, 11.0, 7.0, 3.0, 2..."
Source,object,0.0,16,"[LowestRates, Lower My Bills, Canada Mortgage ..."


In [ ]:
def string_to_cat(df):
    for col in df.columns:
        if is_object_dtype(df[col]) or is_bool_dtype(df[col]):
            df[col] = df[col].astype('category').cat.as_ordered()

def cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1

string_to_cat(Data_Clean)
cat_to_catcode(Data_Clean)
Info(Data_Clean)

,data type,percent missing,No. unique,unique values
JumpstartFlag,int64,0.0,2,"[0, 1]"
Credit_to_Submit,float64,0.0,167,"[3.0, -1.0, 1.0, 14.0, 21.0, 4.0, 10.0, 2.0, 1..."
Lead_to_Submit,float64,0.0,173,"[5.0, -1.0, 74.0, 14.0, 22.0, 7.0, 13.0, 2.0, ..."
Credit FirstStatusDate_Binary,int64,0.0,2,"[1, 0]"
Allocate_to_Credit,float64,0.0,110,"[1.0, -1.0, 0.0, 2.0, 5.0, 41.0, 3.0, 23.0, 10..."
Lead_to_Credit,float64,0.0,302,"[2.0, -1.0, 1.0, 73.0, 8.0, 3.0, 4.0, 0.0, 9.0..."
AllocateDate_Binary,int64,0.0,2,"[1, 0]"
SubmitToLender FirstStatusDate_Binary,int64,0.0,2,"[1, 0]"
Lead_to_Allocate,float64,0.0,564,"[1.0, 8.0, 78.0, -1.0, 10.0, 11.0, 7.0, 3.0, 2..."
Source,int8,0.0,16,"[8, 7, 2, 6, 12, 3, 16, 11, 13, 15, 9, 5, 1, 1..."


#### Allocate Date

In [ ]:
# Creating Baseline Model for Allocate Date
y_Allocate_Date = Data_Clean['AllocateDate_Binary']
x_Allocate_Date = Data_Clean.drop(['LeadDate_Binary','Lead_to_Allocate','AllocateDate_Binary','Lead_to_Credit','Allocate_to_Credit','Credit FirstStatusDate_Binary','Lead_to_Submit','Credit_to_Submit','SubmitToLender FirstStatusDate_Binary','FileComplete FirstStatusDate_Binary'], axis = 1)
x_train_Allocate_Date, x_test_Allocate_Date, y_train_Allocate_Date, y_test_Allocate_Date = train_test_split(x_Allocate_Date, y_Allocate_Date, test_size=0.25, random_state=411)

model_Allocate_Date = LogisticRegression()
model_Allocate_Date.fit(x_train_Allocate_Date,y_train_Allocate_Date)
baseline_prediction_Allocate_Date = model_Allocate_Date.predict(x_test_Allocate_Date)

print(confusion_matrix(baseline_prediction_Allocate_Date,y_test_Allocate_Date))
print(accuracy_score(baseline_prediction_Allocate_Date,y_test_Allocate_Date))
print(f1_score(baseline_prediction_Allocate_Date,y_test_Allocate_Date))

[[4430 6993]
 [   0    0]]
0.38781405935393504
0.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
# Feature Importance
# get importance
importance = model_Allocate_Date.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.10f' % (i,v))

Feature: 0, Score: 0.0000000000
Feature: 1, Score: 0.0000000000
Feature: 2, Score: 0.0000000000
Feature: 3, Score: 0.0000000000
Feature: 4, Score: 0.0000000000
Feature: 5, Score: 0.0000000000
Feature: 6, Score: 0.0000000000


In [ ]:
# Data Creation
Data_Allocate_Date = x_Allocate_Date.copy()
Data_Allocate_Date['AllocateDate_Binary'] = y_Allocate_Date
#correlation
correlation_Allocate_Date = Data_Allocate_Date.corr()
#tick labels
matrix_cols_Allocate_Date = correlation_Allocate_Date.columns.tolist()
#convert to array
corr_array_Allocate_Date  = np.array(correlation_Allocate_Date)
#correlation table
correlation_Allocate_Date

,JumpstartFlag,IsFirstTimeBuyer__c,LoanAmount__c,Loan Amount (50k Bins),LoanPurpose__c,SelfAssessedCreditRating__c,Source,AllocateDate_Binary
JumpstartFlag,1.000000,0.045568,-0.000709,-0.000709,-0.022687,-0.053522,0.103422,0.069046
IsFirstTimeBuyer__c,0.045568,1.000000,-0.002007,-0.002006,-0.390344,-0.147732,0.397219,0.013633
LoanAmount__c,-0.000709,-0.002007,1.000000,1.000000,-0.005035,-0.005674,0.004968,-0.007997
Loan Amount (50k Bins),-0.000709,-0.002006,1.000000,1.000000,-0.005033,-0.005671,0.004964,-0.007998
LoanPurpose__c,-0.022687,-0.390344,-0.005035,-0.005033,1.000000,0.206652,0.020557,0.187541
SelfAssessedCreditRating__c,-0.053522,-0.147732,-0.005674,-0.005671,0.206652,1.000000,-0.194685,0.127110
Source,0.103422,0.397219,0.004968,0.004964,0.020557,-0.194685,1.000000,0.175448
AllocateDate_Binary,0.069046,0.013633,-0.007997,-0.007998,0.187541,0.127110,0.175448,1.000000


In [ ]:
# Creating a Heat Map
#Plotting
trace = go.Heatmap(z = corr_array_Allocate_Date,
                   x = matrix_cols_Allocate_Date,
                   y = matrix_cols_Allocate_Date,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data_Allocate = [trace]
fig = go.Figure(data=data_Allocate,layout=layout)
py.iplot(fig)

In [ ]:
# Creating Model for Allocate Date
y_Allocate_Date_Clean = y_Allocate_Date
x_Allocate_Date_Clean = x_Allocate_Date.drop(['IsFirstTimeBuyer__c','JumpstartFlag','LoanPurpose__c','Loan Amount (50k Bins)'], axis = 1)
x_train_Allocate_Date_Clean, x_test_Allocate_Date_Clean, y_train_Allocate_Date_Clean, y_test_Allocate_Date_Clean = train_test_split(x_Allocate_Date_Clean, y_Allocate_Date_Clean, test_size=0.25, random_state=411)

model_Allocate_Date_Clean = LogisticRegression()
model_Allocate_Date_Clean.fit(x_train_Allocate_Date_Clean,y_train_Allocate_Date_Clean)
baseline_prediction_Allocate_Date_Clean = model_Allocate_Date_Clean.predict(x_test_Allocate_Date_Clean)

print(confusion_matrix(baseline_prediction_Allocate_Date_Clean,y_test_Allocate_Date_Clean))
print(accuracy_score(baseline_prediction_Allocate_Date_Clean,y_test_Allocate_Date_Clean))
print(f1_score(baseline_prediction_Allocate_Date_Clean,y_test_Allocate_Date_Clean))

[[4430 6993]
 [   0    0]]
0.38781405935393504
0.0


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



#### Credit First Date

In [ ]:
# Creating Baseline Model for Credit First
y_Credit_First = Data_Clean['Credit FirstStatusDate_Binary']
x_Credit_First = Data_Clean.drop(['LeadDate_Binary','Lead_to_Credit','Allocate_to_Credit','Credit FirstStatusDate_Binary','Lead_to_Submit','Credit_to_Submit','SubmitToLender FirstStatusDate_Binary','FileComplete FirstStatusDate_Binary'], axis = 1)
x_train_Credit_First, x_test_Credit_First, y_train_Credit_First, y_test_Credit_First = train_test_split(x_Credit_First, y_Credit_First, test_size=0.25, random_state=411)

model_Credit_First = LogisticRegression()
model_Credit_First.fit(x_train_Credit_First,y_train_Credit_First)
baseline_prediction_Credit_First = model_Credit_First.predict(x_test_Credit_First)

print(confusion_matrix(baseline_prediction_Credit_First,y_test_Credit_First))
print(accuracy_score(baseline_prediction_Credit_First,y_test_Credit_First))
print(f1_score(baseline_prediction_Credit_First,y_test_Credit_First ))

[[9026 2308]
 [  79   10]]
0.7910356298695614
0.008309098462816786


In [ ]:
# Feature Importance
# get importance
importance = model_Credit_First.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.10f' % (i,v))

Feature: 0, Score: 0.0000000000
Feature: 1, Score: -0.0000000000
Feature: 2, Score: 0.0000000049
Feature: 3, Score: -0.0000024171
Feature: 4, Score: -0.0000000000
Feature: 5, Score: -0.0000000002
Feature: 6, Score: -0.0000000002
Feature: 7, Score: -0.0000000013
Feature: 8, Score: 0.0000000000


In [ ]:
# Data Creation
Data_Credit_First = x_Credit_First.copy()
Data_Credit_First['Credit FirstStatusDate_Binary'] = y_Credit_First
#correlation
correlation_Credit_First = Data_Credit_First.corr()
#tick labels
matrix_cols_Credit_First = correlation_Credit_First.columns.tolist()
#convert to array
corr_array_Credit_First  = np.array(correlation_Credit_First)
#correlation table
correlation_Credit_First

,JumpstartFlag,IsFirstTimeBuyer__c,LoanAmount__c,Loan Amount (50k Bins),LoanPurpose__c,SelfAssessedCreditRating__c,Source,Lead_to_Allocate,AllocateDate_Binary,Credit FirstStatusDate_Binary
JumpstartFlag,1.000000,0.045568,-0.000709,-0.000709,-0.022687,-0.053522,0.103422,-0.029014,0.069046,0.148894
IsFirstTimeBuyer__c,0.045568,1.000000,-0.002007,-0.002006,-0.390344,-0.147732,0.397219,0.083203,0.013633,-0.035788
LoanAmount__c,-0.000709,-0.002007,1.000000,1.000000,-0.005035,-0.005674,0.004968,-0.001837,-0.007997,-0.003231
Loan Amount (50k Bins),-0.000709,-0.002006,1.000000,1.000000,-0.005033,-0.005671,0.004964,-0.001837,-0.007998,-0.003235
LoanPurpose__c,-0.022687,-0.390344,-0.005035,-0.005033,1.000000,0.206652,0.020557,0.044686,0.187541,0.120822
SelfAssessedCreditRating__c,-0.053522,-0.147732,-0.005674,-0.005671,0.206652,1.000000,-0.194685,0.025302,0.127110,0.033412
Source,0.103422,0.397219,0.004968,0.004964,0.020557,-0.194685,1.000000,0.050752,0.175448,0.110253
Lead_to_Allocate,-0.029014,0.083203,-0.001837,-0.001837,0.044686,0.025302,0.050752,1.000000,0.228611,-0.071035
AllocateDate_Binary,0.069046,0.013633,-0.007997,-0.007998,0.187541,0.127110,0.175448,0.228611,1.000000,0.400896
Credit FirstStatusDate_Binary,0.148894,-0.035788,-0.003231,-0.003235,0.120822,0.033412,0.110253,-0.071035,0.400896,1.000000


In [ ]:
# Creating a Heat Map
#Plotting
trace = go.Heatmap(z = corr_array_Credit_First,
                   x = matrix_cols_Credit_First,
                   y = matrix_cols_Credit_First,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data_Credit = [trace]
fig = go.Figure(data=data_Credit,layout=layout)
py.iplot(fig)

In [ ]:
# Creating Model for Credit First
y_Credit_First_Clean = Data_Clean['Credit FirstStatusDate_Binary']
x_Credit_First_Clean = x_Credit_First.drop(['JumpstartFlag','IsFirstTimeBuyer__c','LoanPurpose__c','AllocateDate_Binary'], axis = 1)
x_train_Credit_First_Clean, x_test_Credit_First_Clean, y_train_Credit_First_Clean, y_test_Credit_First_Clean = train_test_split(x_Credit_First_Clean, y_Credit_First_Clean, test_size=0.25, random_state=411)

model_Credit_First_Clean = LogisticRegression()
model_Credit_First_Clean.fit(x_train_Credit_First_Clean,y_train_Credit_First_Clean)
baseline_prediction_Credit_First_Clean = model_Credit_First_Clean.predict(x_test_Credit_First_Clean)

print(confusion_matrix(baseline_prediction_Credit_First_Clean,y_test_Credit_First_Clean))
print(accuracy_score(baseline_prediction_Credit_First_Clean,y_test_Credit_First_Clean))
print(f1_score(baseline_prediction_Credit_First_Clean,y_test_Credit_First_Clean))

[[9026 2308]
 [  79   10]]
0.7910356298695614
0.008309098462816786


#### Submit to Lender

In [ ]:
# Creating Baseline Model for STL
y_STL = Data_Clean['SubmitToLender FirstStatusDate_Binary']
x_STL = Data_Clean.drop(['LeadDate_Binary','Lead_to_Submit','Credit_to_Submit','SubmitToLender FirstStatusDate_Binary','FileComplete FirstStatusDate_Binary'], axis = 1)
x_train_STL, x_test_STL, y_train_STL, y_test_STL = train_test_split(x_STL, y_STL, test_size=0.25, random_state=411)

model_STL = LogisticRegression()
model_STL.fit(x_train_STL,y_train_STL)
baseline_prediction_STL = model_STL.predict(x_test_STL)

print(confusion_matrix(baseline_prediction_STL,y_test_STL))
print(accuracy_score(baseline_prediction_STL,y_test_STL))
print(f1_score(baseline_prediction_STL,y_test_STL))

[[10909   428]
 [   86     0]]
0.9550030639936969
0.0


In [ ]:
# Feature Importance
# get importance
importance = model_STL.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.10f' % (i,v))

Feature: 0, Score: -0.0000000000
Feature: 1, Score: -0.0000000003
Feature: 2, Score: 0.0000000120
Feature: 3, Score: -0.0000091168
Feature: 4, Score: -0.0000000004
Feature: 5, Score: -0.0000000014
Feature: 6, Score: -0.0000000019
Feature: 7, Score: -0.0000000065
Feature: 8, Score: -0.0000000001
Feature: 9, Score: 0.0000000005
Feature: 10, Score: 0.0000000004
Feature: 11, Score: 0.0000000000


In [ ]:
# Data Creation
Data_STL = x_STL.copy()
Data_STL['SubmitToLender FirstStatusDate_Binary'] = y_STL
#correlation
correlation_STL = Data_STL.corr()
#tick labels
matrix_cols_STL = correlation_STL.columns.tolist()
#convert to array
corr_array_STL  = np.array(correlation_STL)
#correlation table
correlation_STL

,JumpstartFlag,IsFirstTimeBuyer__c,LoanAmount__c,Loan Amount (50k Bins),LoanPurpose__c,SelfAssessedCreditRating__c,Source,Lead_to_Allocate,AllocateDate_Binary,Lead_to_Credit,Allocate_to_Credit,Credit FirstStatusDate_Binary,SubmitToLender FirstStatusDate_Binary
JumpstartFlag,1.000000,0.045568,-0.000709,-0.000709,-0.022687,-0.053522,0.103422,-0.029014,0.069046,-0.002324,0.013628,0.148894,0.055355
IsFirstTimeBuyer__c,0.045568,1.000000,-0.002007,-0.002006,-0.390344,-0.147732,0.397219,0.083203,0.013633,0.015882,-0.005785,-0.035788,-0.050090
LoanAmount__c,-0.000709,-0.002007,1.000000,1.000000,-0.005035,-0.005674,0.004968,-0.001837,-0.007997,-0.000712,-0.000726,-0.003231,-0.001260
Loan Amount (50k Bins),-0.000709,-0.002006,1.000000,1.000000,-0.005033,-0.005671,0.004964,-0.001837,-0.007998,-0.000711,-0.000726,-0.003235,-0.001269
LoanPurpose__c,-0.022687,-0.390344,-0.005035,-0.005033,1.000000,0.206652,0.020557,0.044686,0.187541,0.025920,0.021723,0.120822,0.088243
SelfAssessedCreditRating__c,-0.053522,-0.147732,-0.005674,-0.005671,0.206652,1.000000,-0.194685,0.025302,0.127110,0.013227,0.008872,0.033412,0.007416
Source,0.103422,0.397219,0.004968,0.004964,0.020557,-0.194685,1.000000,0.050752,0.175448,0.009183,0.028803,0.110253,0.048786
Lead_to_Allocate,-0.029014,0.083203,-0.001837,-0.001837,0.044686,0.025302,0.050752,1.000000,0.228611,0.270390,0.002986,-0.071035,-0.038417
AllocateDate_Binary,0.069046,0.013633,-0.007997,-0.007998,0.187541,0.127110,0.175448,0.228611,1.000000,0.088122,0.090052,0.400896,0.157038
Lead_to_Credit,-0.002324,0.015882,-0.000712,-0.000711,0.025920,0.013227,0.009183,0.270390,0.088122,1.000000,0.291939,0.219813,0.054433


In [ ]:
# Creating a Heat Map
#Plotting
trace = go.Heatmap(z = corr_array_STL,
                   x = matrix_cols_STL,
                   y = matrix_cols_STL,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data_STL = [trace]
fig = go.Figure(data=data_STL,layout=layout)
py.iplot(fig)

In [ ]:
# Creating Model for STL
y_STL_Clean = Data_Clean['SubmitToLender FirstStatusDate_Binary']
x_STL_Clean = x_STL.drop(['JumpstartFlag','Credit FirstStatusDate_Binary','IsFirstTimeBuyer__c','Allocate_to_Credit','Lead_to_Credit','AllocateDate_Binary'], axis = 1)
x_train_STL_Clean, x_test_STL_Clean, y_train_STL_Clean, y_test_STL_Clean = train_test_split(x_STL_Clean, y_STL_Clean, test_size=0.25, random_state=411)

model_STL_Clean = LogisticRegression()
model_STL_Clean.fit(x_train_STL_Clean,y_train_STL_Clean)
baseline_prediction_STL_Clean = model_STL_Clean.predict(x_test_STL_Clean)

print(confusion_matrix(baseline_prediction_STL_Clean,y_test_STL_Clean))
print(accuracy_score(baseline_prediction_STL_Clean,y_test_STL_Clean))
print(f1_score(baseline_prediction_STL,y_test_STL_Clean))

[[10911   428]
 [   84     0]]
0.9551781493478071
0.0


#### File Complete

In [ ]:
# Creating Baseline Model for Closing
y_File_Complete = Data_Clean['FileComplete FirstStatusDate_Binary']
x_File_Complete = Data_Clean.drop('FileComplete FirstStatusDate_Binary', axis = 1)
x_train_File_Complete, x_test_File_Complete, y_train_File_Complete, y_test_File_Complete = train_test_split(x_File_Complete, y_File_Complete, test_size=0.25, random_state=411)

model = LogisticRegression()
model.fit(x_train_File_Complete,y_train_File_Complete)
baseline_prediction_File_Complete = model.predict(x_test_File_Complete)

print(confusion_matrix(baseline_prediction_File_Complete,y_test_File_Complete))
print(accuracy_score(baseline_prediction_File_Complete,y_test_File_Complete))
print(f1_score(baseline_prediction_File_Complete,y_test_File_Complete))

[[11160   185]
 [   43    35]]
0.9800402696314453
0.23489932885906037


In [ ]:
# Feature Importance
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.10f' % (i,v))

Feature: 0, Score: -0.0000326189
Feature: 1, Score: -0.0092913616
Feature: 2, Score: -0.0000001171
Feature: 3, Score: -0.0000086283
Feature: 4, Score: -0.0148581336
Feature: 5, Score: -0.0487887537
Feature: 6, Score: -0.0640582503
Feature: 7, Score: -0.0081921218
Feature: 8, Score: -0.2070857680
Feature: 9, Score: -0.0038044847
Feature: 10, Score: 0.0187244315
Feature: 11, Score: 0.0126367605
Feature: 12, Score: 0.0012504639
Feature: 13, Score: 0.1072001174
Feature: 14, Score: 0.0000992545
Feature: 15, Score: 0.0034436478


In [ ]:
#correlation
correlation = Data_Clean.corr()
#tick labels
matrix_cols = correlation.columns.tolist()
#convert to array
corr_array  = np.array(correlation)
#correlation table
correlation

,JumpstartFlag,IsFirstTimeBuyer__c,LoanAmount__c,Loan Amount (50k Bins),LoanPurpose__c,SelfAssessedCreditRating__c,Source,LeadDate_Binary,Lead_to_Allocate,AllocateDate_Binary,Lead_to_Credit,Allocate_to_Credit,Credit FirstStatusDate_Binary,Lead_to_Submit,Credit_to_Submit,SubmitToLender FirstStatusDate_Binary,FileComplete FirstStatusDate_Binary
JumpstartFlag,1.000000,0.045568,-0.000709,-0.000709,-0.022687,-0.053522,0.103422,NaN,-0.029014,0.069046,-0.002324,0.013628,0.148894,0.009098,0.021986,0.055355,0.042935
IsFirstTimeBuyer__c,0.045568,1.000000,-0.002007,-0.002006,-0.390344,-0.147732,0.397219,NaN,0.083203,0.013633,0.015882,-0.005785,-0.035788,-0.000126,-0.002871,-0.050090,-0.039110
LoanAmount__c,-0.000709,-0.002007,1.000000,1.000000,-0.005035,-0.005674,0.004968,NaN,-0.001837,-0.007997,-0.000712,-0.000726,-0.003231,-0.000640,-0.000159,-0.001260,-0.000917
Loan Amount (50k Bins),-0.000709,-0.002006,1.000000,1.000000,-0.005033,-0.005671,0.004964,NaN,-0.001837,-0.007998,-0.000711,-0.000726,-0.003235,-0.000643,-0.000159,-0.001269,-0.000918
LoanPurpose__c,-0.022687,-0.390344,-0.005035,-0.005033,1.000000,0.206652,0.020557,NaN,0.044686,0.187541,0.025920,0.021723,0.120822,0.012849,0.005762,0.088243,0.070783
SelfAssessedCreditRating__c,-0.053522,-0.147732,-0.005674,-0.005671,0.206652,1.000000,-0.194685,NaN,0.025302,0.127110,0.013227,0.008872,0.033412,0.001995,-0.008325,0.007416,-0.008113
Source,0.103422,0.397219,0.004968,0.004964,0.020557,-0.194685,1.000000,NaN,0.050752,0.175448,0.009183,0.028803,0.110253,0.023399,0.013233,0.048786,0.045588
LeadDate_Binary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lead_to_Allocate,-0.029014,0.083203,-0.001837,-0.001837,0.044686,0.025302,0.050752,NaN,1.000000,0.228611,0.270390,0.002986,-0.071035,0.022933,-0.004195,-0.038417,-0.027921
AllocateDate_Binary,0.069046,0.013633,-0.007997,-0.007998,0.187541,0.127110,0.175448,NaN,0.228611,1.000000,0.088122,0.090052,0.400896,0.079671,0.019674,0.157038,0.113522


In [ ]:
# Creating a Heat Map
#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data_final = [trace]
fig = go.Figure(data=data_final,layout=layout)
py.iplot(fig)

In [ ]:
# Creating Model for Closing
y_File_Complete_Clean = Data_Clean['FileComplete FirstStatusDate_Binary']
x_File_Complete_Clean = x_File_Complete.drop(['SubmitToLender FirstStatusDate_Binary','Credit_to_Submit','JumpstartFlag','LoanAmount__c','Loan Amount (50k Bins)'], axis = 1)
x_train_File_Complete_Clean, x_test_File_Complete_Clean, y_train_File_Complete_Clean, y_test_File_Complete_Clean = train_test_split(x_File_Complete_Clean, y_File_Complete_Clean, test_size=0.25, random_state=411)

model_Clean = LogisticRegression()
model_Clean.fit(x_train_File_Complete_Clean,y_train_File_Complete_Clean)
baseline_prediction_File_Complete_Clean = model_Clean.predict(x_test_File_Complete_Clean)

print(confusion_matrix(baseline_prediction_File_Complete_Clean,y_test_File_Complete_Clean))
print(accuracy_score(baseline_prediction_File_Complete_Clean,y_test_File_Complete_Clean))
print(f1_score(baseline_prediction_File_Complete_Clean,y_test_File_Complete_Clean))

[[11179   199]
 [   24    21]]
0.9804779830167206
0.15849056603773584


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

